In [ ]:
import pandas as pd
import numpy as np
from flask import Flask, render_template, jsonify

In [3]:
circuits_df = pd.read_csv("datasets/circuits.csv")
circuits_df

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,\N,http://en.wikipedia.org/wiki/Losail_Internatio...


In [9]:
constructor_results_df = pd.read_csv('datasets/constructor_results.csv')
constructor_results_df

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,\N
1,2,18,2,8.0,\N
2,3,18,3,9.0,\N
3,4,18,4,5.0,\N
4,5,18,5,2.0,\N
...,...,...,...,...,...
12285,16785,1110,214,10.0,\N
12286,16786,1110,213,1.0,\N
12287,16787,1110,51,0.0,\N
12288,16788,1110,3,0.0,\N


In [11]:
constructor_standings_df = pd.read_csv('datasets/constructor_standings.csv')
constructor_standings_df

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0
...,...,...,...,...,...,...,...
13046,28568,1110,214,57.0,6,6,0
13047,28569,1110,3,11.0,7,7,0
13048,28570,1110,213,3.0,10,10,0
13049,28571,1110,210,11.0,8,8,0


In [14]:
constructors_df = pd.read_csv('datasets/constructors.csv')
constructors_df

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,213,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri


In [41]:
results = pd.read_csv('datasets/results.csv')
races = pd.read_csv('datasets/races.csv')
drivers = pd.read_csv('datasets/drivers.csv')
constructors = pd.read_csv('datasets/constructors.csv')

# Merge datasets to get required data
# data_merged = results.merge(races, on='raceId').merge(drivers, on='driverId').merge(constructors, on='constructorId', )
data_merged = results.merge(races[['raceId', 'year']], on='raceId', suffixes=('', '_race'))
data_merged = data_merged.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', suffixes=('', '_driver'))
data_merged = data_merged.merge(constructors[['constructorId', 'name']], on='constructorId', suffixes=('', '_constructor'))

# Filter data for years 2010 onwards
data_merged = data_merged[data_merged['year'] >= 2010]

# Aggregate points by driver across all years and select top 10 drivers
top_drivers = data_merged.groupby(['driverId', 'forename', 'surname'])['points'].sum().nlargest(10).reset_index()['driverId']

# Filter the merged data to include only top 10 drivers
data_merged = data_merged[data_merged['driverId'].isin(top_drivers)]

In [42]:
data_merged.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'forename', 'surname', 'name'],
      dtype='object')

In [43]:
data_merged

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,forename,surname,name
52,20325,337,1,1,2,4,3,3,3,15.0,...,5983578,42,4,1:59.560,189.665,1,2010,Lewis,Hamilton,McLaren
53,20352,338,1,1,2,11,6,6,6,8.0,...,5646429,47,3,1:28.506,215.700,1,2010,Lewis,Hamilton,McLaren
54,20376,339,1,1,2,20,6,6,6,8.0,...,5651883,32,4,1:37.745,204.151,1,2010,Lewis,Hamilton,McLaren
55,20396,340,1,1,2,6,2,2,2,18.0,...,6403693,13,1,1:42.061,192.273,1,2010,Lewis,Hamilton,McLaren
56,20432,341,1,1,2,3,14,14,14,0.0,...,\N,59,1,1:24.357,198.655,29,2010,Lewis,Hamilton,McLaren
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25816,24923,1045,815,211,11,5,18,18,18,0.0,...,\N,40,9,1:33.629,208.089,5,2020,Sergio,Pérez,Racing Point
25817,24926,1046,815,211,11,5,1,1,1,25.0,...,5475114,69,3,0:56.789,224.599,1,2020,Sergio,Pérez,Racing Point
25818,24965,1047,815,211,11,19,\N,R,20,0.0,...,\N,6,19,1:43.263,193.625,7,2020,Sergio,Pérez,Racing Point
25856,26058,1109,817,213,3,13,13,13,13,0.0,...,\N,67,17,1:23.934,187.904,11,2023,Daniel,Ricciardo,AlphaTauri


In [44]:
data_sankey = data_merged.groupby(['forename', 'surname', 'name', 'year'])['points'].sum().reset_index()
data_sankey['source'] = data_sankey['forename'] + ' ' + data_sankey['surname']
data_sankey['target'] = data_sankey['name']
data_sankey['value'] = data_sankey['points']
nodes = list(set(data_sankey['source'].tolist() + data_sankey['target'].tolist()))
links = [
    {"source": nodes.index(row['source']), "target": nodes.index(row['target']), "value": row['value'], "year": row['year']}
    for index, row in data_sankey.iterrows()
]

In [49]:
data_sankey[data_sankey['surname']=='Vettel']

,forename,surname,name,year,points,source,target,value
72,Sebastian,Vettel,Aston Martin,2021,43.0,Sebastian Vettel,Aston Martin,43.0
73,Sebastian,Vettel,Aston Martin,2022,37.0,Sebastian Vettel,Aston Martin,37.0
74,Sebastian,Vettel,Ferrari,2015,278.0,Sebastian Vettel,Ferrari,278.0
75,Sebastian,Vettel,Ferrari,2016,212.0,Sebastian Vettel,Ferrari,212.0
76,Sebastian,Vettel,Ferrari,2017,317.0,Sebastian Vettel,Ferrari,317.0
77,Sebastian,Vettel,Ferrari,2018,320.0,Sebastian Vettel,Ferrari,320.0
78,Sebastian,Vettel,Ferrari,2019,240.0,Sebastian Vettel,Ferrari,240.0
79,Sebastian,Vettel,Ferrari,2020,33.0,Sebastian Vettel,Ferrari,33.0
80,Sebastian,Vettel,Red Bull,2010,256.0,Sebastian Vettel,Red Bull,256.0
81,Sebastian,Vettel,Red Bull,2011,392.0,Sebastian Vettel,Red Bull,392.0


In [2]:
import pandas as pd

# Load datasets
drivers = pd.read_csv('datasets/drivers.csv')
results = pd.read_csv('datasets/results.csv')
races = pd.read_csv('datasets/races.csv')
constructors = pd.read_csv('datasets/constructors.csv')

# Merge drivers with results
driver_results = pd.merge(drivers, results, on='driverId')

# Merge driver_results with races to get the year of the race
driver_results = pd.merge(driver_results, races[['raceId', 'year']], on='raceId')

# Group by driver and aggregate necessary metrics
driver_aggregates = driver_results.groupby(['driverId', 'forename', 'surname', 'nationality']).agg({
    'raceId': pd.Series.nunique,
    'positionOrder': lambda x: (x == 1).sum(),
    'points': 'sum',
    'rank': lambda x: (x == '1').sum()
}).rename(columns={'raceId': 'No. of Races Played', 'positionOrder': 'No. of Wins', 'points': 'Total Points', 'rank': 'No. of Fastest Laps'})

# Calculate podium finishes
podiums = driver_results[driver_results['positionOrder'].isin([1, 2, 3])]
podium_summary = podiums.pivot_table(index='driverId', columns='positionOrder', aggfunc='size', fill_value=0).rename(columns={1: '1st', 2: '2nd', 3: '3rd'})

# Calculate pole positions
pole_positions = driver_results[driver_results['grid'] == 1].groupby('driverId')['raceId'].nunique().rename('Pole Positions')

# Join all computed series and dataframes into the main summary
driver_summary = driver_aggregates.join(podium_summary, how='left')
driver_summary = driver_summary.join(pole_positions, how='left')

# Calculate years active
years_active = driver_results.groupby('driverId')['year'].agg(['min', 'max']).apply(lambda x: f"{x['min']} - {'Current' if x['max'] == pd.Timestamp.now().year else x['max']}", axis=1).rename('Years Active')
driver_summary = driver_summary.join(years_active, how='left').reset_index()

# Concatenate names to create full name
driver_summary['Name of the Driver'] = driver_summary['forename'] + ' ' + driver_summary['surname']
driver_summary.drop(['forename', 'surname'], axis=1, inplace=True)

# Reorder columns for final output
driver_summary = driver_summary[['Name of the Driver', 'nationality', 'Years Active', 'No. of Races Played', 'No. of Wins', '1st', '2nd', '3rd', 'Pole Positions', 'Total Points', 'No. of Fastest Laps']]

driver_summary

,Name of the Driver,nationality,Years Active,No. of Races Played,No. of Wins,1st,2nd,3rd,Pole Positions,Total Points,No. of Fastest Laps
0,Lewis Hamilton,British,2007 - 2023,322,103,103.0,55.0,37.0,104.0,4540.5,63
1,Nick Heidfeld,German,2000 - 2011,184,0,0.0,8.0,5.0,1.0,259.0,2
2,Nico Rosberg,German,2006 - 2016,206,23,23.0,25.0,9.0,30.0,1594.5,20
3,Fernando Alonso,Spanish,2001 - 2023,370,32,32.0,39.0,33.0,22.0,2203.0,22
4,Heikki Kovalainen,Finnish,2007 - 2013,112,1,1.0,2.0,1.0,1.0,105.0,2
...,...,...,...,...,...,...,...,...,...,...,...
852,Mick Schumacher,German,2021 - 2022,44,0,NaN,NaN,NaN,NaN,12.0,0
853,Guanyu Zhou,Chinese,2022 - 2023,34,0,NaN,NaN,NaN,NaN,10.0,2
854,Nyck de Vries,Dutch,2022 - 2023,11,0,NaN,NaN,NaN,NaN,2.0,0
855,Oscar Piastri,Australian,2023 - 2023,12,0,NaN,NaN,NaN,NaN,27.0,0


In [1]:
import pandas as pd

# Load datasets
drivers = pd.read_csv('datasets/drivers.csv')
results = pd.read_csv('datasets/results.csv')
races = pd.read_csv('datasets/races.csv')
constructors = pd.read_csv('datasets/constructors.csv')

# Merge drivers with results
driver_results = pd.merge(drivers, results, on='driverId')

# Include constructor information
driver_results = pd.merge(driver_results, constructors[['constructorId', 'constructorRef', 'name']], on='constructorId')

# Merge driver_results with races to get the year of the race
driver_results = pd.merge(driver_results, races[['raceId', 'year']], on='raceId')

# Group by driver and aggregate necessary metrics, keeping driver names
driver_aggregates = driver_results.groupby(['driverId', 'forename', 'surname', 'nationality']).agg({
    'raceId': 'nunique',
    'positionOrder': lambda x: (x == 1).sum(),
    'points': 'sum',
    'rank': lambda x: (x == '1').sum()
}).rename(columns={'raceId': 'No. of Races Played', 'positionOrder': 'No. of Wins', 'points': 'Total Points', 'rank': 'No. of Fastest Laps'}).reset_index()

# Calculate podium finishes
podiums = driver_results[driver_results['positionOrder'].isin([1, 2, 3])]
podium_summary = podiums.pivot_table(index='driverId', columns='positionOrder', aggfunc='size', fill_value=0).rename(columns={1: '1st', 2: '2nd', 3: '3rd'})

# Calculate pole positions
pole_positions = driver_results[driver_results['grid'] == 1].groupby('driverId')['raceId'].nunique().rename('Pole Positions')

# Join all computed series and dataframes into the main summary
driver_summary = driver_aggregates.set_index('driverId').join(podium_summary, how='left')
driver_summary = driver_summary.join(pole_positions, how='left')

# Get the most recent constructor for each driver
latest_constructor = driver_results.loc[driver_results.groupby('driverId')['raceId'].idxmax(), ['driverId', 'name']]
latest_constructor.set_index('driverId', inplace=True)

# Calculate years active
years_active = driver_results.groupby('driverId')['year'].agg(['min', 'max']).apply(lambda x: f"{x['min']} - {'Current' if x['max'] == pd.Timestamp.now().year-1 else x['max']}", axis=1).rename('Years Active')
driver_summary = driver_summary.join(years_active, how='left')

# Join latest constructor information
driver_summary = driver_summary.join(latest_constructor, how='left').rename(columns={'name': 'Current Team'})

# Concatenate names to create full name
driver_summary['Name of the Driver'] = driver_summary['forename'] + ' ' + driver_summary['surname']
driver_summary.drop(['forename', 'surname'], axis=1, inplace=True)

# Calculate total podiums
driver_summary['Total Podiums'] = driver_summary[['1st', '2nd', '3rd']].sum(axis=1)

# Reorder columns for final output
driver_summary = driver_summary[['Name of the Driver', 'nationality', 'Current Team', 'Years Active', 'No. of Races Played', 'No. of Wins', '1st', '2nd', '3rd', 'Total Podiums', 'Pole Positions', 'Total Points', 'No. of Fastest Laps']]

driver_summary

,Name of the Driver,nationality,Current Team,Years Active,No. of Races Played,No. of Wins,1st,2nd,3rd,Total Podiums,Pole Positions,Total Points,No. of Fastest Laps
driverId,,,,,,,,,,,,,
1,Lewis Hamilton,British,Mercedes,2007 - Current,322,103,103.0,55.0,37.0,195.0,104.0,4540.5,63
2,Nick Heidfeld,German,Renault,2000 - 2011,184,0,0.0,8.0,5.0,13.0,1.0,259.0,2
3,Nico Rosberg,German,Mercedes,2006 - 2016,206,23,23.0,25.0,9.0,57.0,30.0,1594.5,20
4,Fernando Alonso,Spanish,Aston Martin,2001 - Current,370,32,32.0,39.0,33.0,104.0,22.0,2203.0,22
5,Heikki Kovalainen,Finnish,Lotus F1,2007 - 2013,112,1,1.0,2.0,1.0,4.0,1.0,105.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,Mick Schumacher,German,Haas F1 Team,2021 - 2022,44,0,NaN,NaN,NaN,0.0,NaN,12.0,0
855,Guanyu Zhou,Chinese,Alfa Romeo,2022 - Current,34,0,NaN,NaN,NaN,0.0,NaN,10.0,2
856,Nyck de Vries,Dutch,AlphaTauri,2022 - Current,11,0,NaN,NaN,NaN,0.0,NaN,2.0,0


In [3]:
import pandas as pd

# Load datasets
constructor_standings = pd.read_csv('datasets/constructor_standings.csv')
constructors = pd.read_csv('datasets/constructors.csv')
driver_standings = pd.read_csv('datasets/driver_standings.csv')
drivers = pd.read_csv('datasets/drivers.csv')
results = pd.read_csv('datasets/results.csv')
races = pd.read_csv('datasets/races.csv')

# Calculating games played per constructor (races attended)
races_per_constructor = results['constructorId'].value_counts().reset_index()
races_per_constructor.columns = ['constructorId', 'NoOfGamesPlayed']

# Counting drivers per constructor
drivers_per_constructor = results[['constructorId', 'driverId']].drop_duplicates()
drivers_per_constructor_count = drivers_per_constructor.groupby('constructorId').count().reset_index()
drivers_per_constructor_count.columns = ['constructorId', 'NoOfDrivers']

# Best driver in history by total wins per constructor
results_with_drivers = results.merge(drivers, on='driverId')
driver_wins = results_with_drivers[results_with_drivers['positionOrder'] == 1]
best_driver_per_constructor = driver_wins.groupby('constructorId').agg({'driverId':'max'}).reset_index()
best_driver_per_constructor.columns = ['constructorId', 'BestDriverId']
best_driver_names = drivers[['driverId', 'surname']].rename(columns={'surname': 'BestDriver'})
best_driver_per_constructor = best_driver_per_constructor.merge(best_driver_names, left_on='BestDriverId', right_on='driverId', how='left')

# Number of wins per constructor
constructor_wins = driver_wins.groupby('constructorId').size().reset_index(name='NoOfWins')

# Podium finishes per constructor
podium_finishes = results_with_drivers[results_with_drivers['positionOrder'].isin([1, 2, 3])]
podium_counts = podium_finishes.pivot_table(index='constructorId', columns='positionOrder', aggfunc='size', fill_value=0).reset_index()
podium_counts.columns = ['constructorId', 'FirstPlace', 'SecondPlace', 'ThirdPlace']

# Pole positions count (Assuming position 1 in qualifying is pole)
pole_positions = results_with_drivers[results_with_drivers['grid'] == 1]
pole_positions_count = pole_positions.groupby('constructorId').size().reset_index(name='PolePositions')

# Total points per constructor
total_points = constructor_standings.groupby('constructorId')['points'].sum().reset_index()

# Combine all data
constructor_details = constructors[['constructorId', 'name', 'nationality']].merge(races_per_constructor, on='constructorId')
constructor_details = constructor_details.merge(drivers_per_constructor_count, on='constructorId')
constructor_details = constructor_details.merge(best_driver_per_constructor[['constructorId', 'BestDriver']], on='constructorId')
constructor_details = constructor_details.merge(constructor_wins, on='constructorId', how='left')
constructor_details = constructor_details.merge(podium_counts, on='constructorId', how='left')
constructor_details = constructor_details.merge(pole_positions_count, on='constructorId', how='left')
constructor_details = constructor_details.merge(total_points, on='constructorId', how='left')

# Clean and fill NaNs with zeros for counts and sums
constructor_details.fillna(0, inplace=True)

constructor_details

,constructorId,name,nationality,NoOfGamesPlayed,NoOfDrivers,BestDriver,NoOfWins,FirstPlace,SecondPlace,ThirdPlace,PolePositions,points
0,1,McLaren,British,1855,55,Ricciardo,179,179,157,144,156.0,57557.5
1,2,BMW Sauber,German,140,4,Kubica,1,1,10,6,1.0,2932.0
2,3,Williams,British,1609,61,Maldonado,114,114,113,87,128.0,32147.5
3,4,Renault,French,787,26,Jabouille,35,35,35,33,51.0,17168.0
4,5,Toro Rosso,Italian,536,14,Vettel,1,1,1,1,1.0,5193.0
5,6,Ferrari,Italian,2371,97,Leclerc,243,243,293,277,249.0,91389.0
6,9,Red Bull,Austrian,720,12,Verstappen,104,104,75,74,90.0,68243.5
7,11,Honda,Japanese,152,8,Ginther,3,3,1,5,2.0,1062.0
8,17,Jordan,Irish,500,30,Hill,4,4,4,11,2.0,2502.0
9,22,Benetton,Italian,520,17,Nannini,27,27,32,43,15.0,7349.5


In [7]:
import pandas as pd

# Load datasets
circuits = pd.read_csv('datasets/circuits.csv')
races = pd.read_csv('datasets/races.csv')
results = pd.read_csv('datasets/results.csv')
lap_times = pd.read_csv('datasets/lap_times.csv')
pit_stops = pd.read_csv('datasets/pit_stops.csv')

In [8]:
# Merge the necessary datasets
races_circuits = races.merge(circuits, on='circuitId', how='left')
race_results = races_circuits.merge(results, on='raceId', how='left')
race_laptimes = race_results.merge(lap_times, on=['raceId', 'driverId'], how='left')
final_dataset = race_laptimes.merge(pit_stops, on=['raceId', 'driverId', 'lap'], how='left')

# Convert columns to numeric where necessary
final_dataset['milliseconds_x'] = pd.to_numeric(final_dataset['milliseconds_x'], errors='coerce')
final_dataset['milliseconds_y'] = pd.to_numeric(final_dataset['milliseconds_y'], errors='coerce')
final_dataset['fastestLapSpeed'] = pd.to_numeric(final_dataset['fastestLapSpeed'], errors='coerce')

# Calculate new columns
final_dataset['lapTime_sec'] = final_dataset['milliseconds_x'] / 1000  # from lap_times
final_dataset['pitDuration_sec'] = final_dataset['milliseconds_y'] / 1000  # from pit_stops
final_dataset['fastestLap_mph'] = final_dataset['fastestLapSpeed']  # already in correct format but converted to ensure no errors

# Group by circuit and aggregate data
circuit_details = final_dataset.groupby('name_x').agg({
    'raceId': 'nunique',  # count unique races
    'lapTime_sec': 'mean',
    'grid': 'mean',
    'fastestLap_mph': 'mean',
    'stop': 'count',
    'pitDuration_sec': 'mean',
    'positionOrder': 'mean',
    'points': 'sum'
}).reset_index()

circuit_details.columns = [
    'Circuit Name', 'Total Matches', 'Lap Time (sec)', 'Grid Position',
    'Fastest Lap (mph)', 'Pit Stops', 'Pit Duration (sec)',
    'Finish Position', 'Total Points'
]

# Select the top 20-25 rows based on Total Points
top_circuits = circuit_details.sort_values('Total Points', ascending=False).head(25)

print(top_circuits)

C:\Users\bindu\AppData\Local\Temp\ipykernel_67504\1269209126.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'time_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_dataset = race_laptimes.merge(pit_stops, on=['raceId', 'driverId', 'lap'], how='left')


                Circuit Name  Total Matches  Lap Time (sec)  Grid Position  \
33         Monaco Grand Prix             69     6688.628221      10.549137   
21      Hungarian Grand Prix             38     6212.932268      10.656375   
45        Spanish Grand Prix             53     5739.241838      10.657695   
11       Canadian Grand Prix             52     6268.259374      10.860602   
8       Brazilian Grand Prix             48     6069.120602      10.829124   
9         British Grand Prix             74     5628.080170      10.728962   
3      Australian Grand Prix             37     5773.755659      10.521533   
24        Italian Grand Prix             74     4851.888418      10.768465   
6         Bahrain Grand Prix             19     5925.948881      10.579055   
4        Austrian Grand Prix             36     5269.409567      10.578705   
25       Japanese Grand Prix             37     5697.440195      10.855188   
1       Abu Dhabi Grand Prix             15     5869.233458     

In [9]:
# Merge relevant datasets
df = results.merge(races, on='raceId')
df = df.merge(circuits, left_on='circuitId', right_on='circuitId')
df = df.merge(lap_times, on=['raceId', 'driverId'])
df = df.merge(pit_stops, on=['raceId', 'driverId', 'lap'], how='left')

# Convert milliseconds to seconds for lap times and pit stop durations
df['Lap Time (sec)'] = df['milliseconds_x'] / 1000
df['Pit Duration (sec)'] = df['milliseconds_y'] / 1000

# Create the required dataset with specified columns
final_dataset = df[['name_x', 'grid', 'fastestLapSpeed', 'positionOrder', 'points', 'Lap Time (sec)', 'Pit Duration (sec)']]
final_dataset.rename(columns={
    'name_x': 'Circuit name',
    'grid': 'Grid Position',
    'fastestLapSpeed': 'Fastest Lap (mph)',
    'positionOrder': 'Finish Position',
}, inplace=True)

# Group by Circuit name to count the number of matches (races)
final_dataset['Matches'] = final_dataset.groupby('Circuit name')['Circuit name'].transform('count')

# Count the number of pit stops for each entry
final_dataset['Pit Stops'] = df.groupby(['raceId', 'driverId'])['stop'].transform('count')

final_dataset

C:\Users\bindu\AppData\Local\Temp\ipykernel_67504\2782135207.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'time_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(pit_stops, on=['raceId', 'driverId', 'lap'], how='left')


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [16]:
# Merge relevant datasets
races = races[races['year'] >= 2011]
df = results.merge(races, on='raceId')
df = df.merge(circuits, left_on='circuitId', right_on='circuitId')
df = df.merge(lap_times, on=['raceId', 'driverId'])
df = df.merge(pit_stops, on=['raceId', 'driverId', 'lap'], how='left')

# Convert 'milliseconds_x' and 'milliseconds_y' to numeric, handling non-numeric data
df['milliseconds_x'] = pd.to_numeric(df['milliseconds_x'], errors='coerce')
df['milliseconds_y'] = pd.to_numeric(df['milliseconds_y'], errors='coerce')

# Convert milliseconds to seconds for lap times and pit stop durations
df['Lap Time (sec)'] = df['milliseconds_x'] / 1000
df['Pit Duration (sec)'] = df['milliseconds_y'] / 1000

# Count occurrences of each circuit and filter
circuit_counts = df['name_x'].value_counts()
circuits_to_keep = circuit_counts[circuit_counts >= 5000].index
df = df[df['name_x'].isin(circuits_to_keep)]

# Create the required dataset with specified columns
final_dataset = df[['name_x', 'grid', 'fastestLapSpeed', 'positionOrder', 'points', 'Lap Time (sec)', 'Pit Duration (sec)']]
final_dataset.rename(columns={
    'name_x': 'Circuit name',
    'grid': 'Grid Position',
    'fastestLapSpeed': 'Fastest Lap (mph)',
    'positionOrder': 'Finish Position',
}, inplace=True)

# Group by Circuit name to count the number of matches (races)
final_dataset['Matches'] = final_dataset.groupby('Circuit name')['Circuit name'].transform('count')

# Count the number of pit stops for each entry
final_dataset['Pit Stops'] = df.groupby(['raceId', 'driverId'])['stop'].transform('count')

C:\Users\bindu\AppData\Local\Temp\ipykernel_67504\1798169683.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'time_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(pit_stops, on=['raceId', 'driverId', 'lap'], how='left')
C:\Users\bindu\AppData\Local\Temp\ipykernel_67504\1798169683.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset.rename(columns={
C:\Users\bindu\AppData\Local\Temp\ipykernel_67504\1798169683.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset['Mat

In [17]:
final_dataset

,Circuit name,Grid Position,Fastest Lap (mph),Finish Position,points,Lap Time (sec),Pit Duration (sec),Matches,Pit Stops
0,Australian Grand Prix,1,212.488,1,25.0,5370.259,98.109,10872,2
1,Australian Grand Prix,1,212.488,1,25.0,5370.259,93.006,10872,2
2,Australian Grand Prix,1,212.488,1,25.0,5370.259,92.713,10872,2
3,Australian Grand Prix,1,212.488,1,25.0,5370.259,92.803,10872,2
4,Australian Grand Prix,1,212.488,1,25.0,5370.259,92.342,10872,2
...,...,...,...,...,...,...,...,...,...
263626,Azerbaijan Grand Prix,18,198.663,20,0.0,NaN,108.948,5403,0
263627,Azerbaijan Grand Prix,18,198.663,20,0.0,NaN,109.337,5403,0
263628,Azerbaijan Grand Prix,18,198.663,20,0.0,NaN,109.118,5403,0
263629,Azerbaijan Grand Prix,18,198.663,20,0.0,NaN,109.353,5403,0


In [20]:
final_dataset['Circuit name'].value_counts()

21